# Project ADA - Where to eat in Chicago?
## Group prADA

### Description


### 1. Getting a sense of the dataset - Chicagoan health food inspections 

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import math

from requests import get
from bs4 import BeautifulSoup

In [2]:
fi_df = pd.read_csv('./Data/food-inspections.csv', sep = ',', header = 0, names = ['inspection id', 'restaurant name DBA', 'restaurant name AKA', 'license numer','facility type','risk','address','city','state','zip','inspection date','inspection type','result','violations','latitude','longitude','location','historical','zip codes','commun. areas','census tr.','wards'])


First, we need to check which columns are usable, a preview on the file hints that some columns might be completely empty as it is the case for the following:

In [3]:
fi_df['historical'].value_counts() #same for  columns: zip codes, community areas, census tracts and wards

Series([], Name: historical, dtype: int64)

So we simply remove them since they are of no use for us.

In [4]:
fi_df=fi_df.drop(columns=['historical','zip codes','commun. areas','census tr.','wards'])
fi_df.head(10)

,inspection id,restaurant name DBA,restaurant name AKA,license numer,facility type,risk,address,city,state,zip,inspection date,inspection type,result,violations,latitude,longitude,location
0,2321132,HOST INTERNATIONAL INC,VIENNA BEEF/PIZZERIA UNO (T1 C6),34222.0,Restaurant,Risk 2 (Medium),11601 W TOUHY AVE,CHICAGO,IL,60666.0,2019-11-06T00:00:00.000,Complaint Re-Inspection,Pass,NaN,42.008536,-87.914428,"{'longitude': '42.008536400868735', 'latitude'..."
1,2321127,STARBUCKS,STARBUCKS (T1 C-07),2583234.0,Restaurant,Risk 2 (Medium),11601 W TOUHY AVE,CHICAGO,IL,60666.0,2019-11-06T00:00:00.000,Complaint Re-Inspection,Pass,NaN,42.008536,-87.914428,"{'longitude': '42.008536400868735', 'latitude'..."
2,2321168,DUNKIN DONUTS,DUNKIN DONUTS,15623.0,Restaurant,Risk 2 (Medium),5200 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-11-06T00:00:00.000,Complaint Re-Inspection,Pass,NaN,41.975913,-87.692494,"{'longitude': '41.97591289899352', 'latitude':..."
3,2321165,LA HUMITA,LA HUMITA,1622351.0,Restaurant,Risk 1 (High),3466 N PULASKI RD,CHICAGO,IL,60641.0,2019-11-06T00:00:00.000,Canvass,No Entry,NaN,41.944678,-87.727496,"{'longitude': '41.94467795956372', 'latitude':..."
4,2321129,MING BRICKYARD SUPER CHINA BUFFET INC.,SUPER CHINA BUFFET,1490057.0,Restaurant,Risk 1 (High),2650 N NARRAGANSETT AVE,CHICAGO,IL,60639.0,2019-11-06T00:00:00.000,Complaint,Pass,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...",41.928847,-87.785781,"{'longitude': '41.928847223323764', 'latitude'..."
5,2321196,"SODEXO AMERICA, LLC",UNITED EMPLOYEE CAFETERIA (T1-C MID LEVEL),2560546.0,Restaurant,Risk 1 (High),11601 W TOUHY AVE,CHICAGO,IL,60666.0,2019-11-06T00:00:00.000,Canvass,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,42.008536,-87.914428,"{'longitude': '42.008536400868735', 'latitude'..."
6,2321148,WILLIAM G. HIBBARD ELEMENTARY SCHOOL,WILLIAM G HIBBARD ELEMENTARY SCHOOL,23801.0,School,Risk 1 (High),3244 W Ainslie ST,CHICAGO,IL,60625.0,2019-11-06T00:00:00.000,Canvass,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.970363,-87.710362,"{'longitude': '41.97036267439323', 'latitude':..."
7,2321187,BANG BANG BISCUITS,BANG BANG BISCUITS,2458777.0,Restaurant,Risk 1 (High),4947 N DAMEN AVE,CHICAGO,IL,60625.0,2019-11-06T00:00:00.000,Canvass,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.971969,-87.679136,"{'longitude': '41.971969426617925', 'latitude'..."
8,2321166,SUBWAY,SUBWAY,2529116.0,Restaurant,Risk 1 (High),2620 N NARRAGANSETT AVE,CHICAGO,IL,60639.0,2019-11-06T00:00:00.000,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.927995,-87.785752,"{'longitude': '41.92799528871574', 'latitude':..."
9,2321116,Scammon Elementary School,Scammon Elementary School,25241.0,School,Risk 1 (High),4201 W Henderson (3332N),CHICAGO,IL,60641.0,2019-11-06T00:00:00.000,Canvass,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.941649,-87.732229,"{'longitude': '41.941649437310005', 'latitude'..."


Our research focuses on the city Chicago, our dataset however contains numerous other locations which we have to discard.

In [5]:
fi_df['city']=fi_df['city'].apply(lambda c: str(c).upper())

In [6]:
fi_df['city']=fi_df['city'].replace({'CCHICAGO':'CHICAGO'})

In [7]:
fi_df['city'].value_counts()

CHICAGO               195083
NAN                      139
SCHAUMBURG                25
MAYWOOD                   16
ELK GROVE VILLAGE         13
                       ...  
COUNTRY CLUB HILLS         1
FRANKFORT                  1
WHEATON                    1
BLOOMINGDALE               1
GLENCOE                    1
Name: city, Length: 66, dtype: int64

In [8]:
fi_df=fi_df[fi_df['city']=='CHICAGO']
fi_df['city'].value_counts()

CHICAGO    195083
Name: city, dtype: int64

Everything is clean now on that level. Next we consider the facility types, we are in the skin of a tourist or any other person looking for a place to eat in Chicago. So the facility types such as churches, schools, hospitals etc. are not taken into consideration.

In [9]:
#1. drop all nan values
fi_df=fi_df.dropna(subset=['facility type'])

#2. drop all values that aren't litterals
fi_df['facility type']=fi_df['facility type'].astype(str)

non_numerical_ft=fi_df['facility type'].apply(lambda f: not f.isdigit())
fi_df=fi_df[non_numerical_ft]

#3. keep only facility types of interest
#discarded_facilities=['care','hospital','church','gas','children','kiosk','store','school','kitchen','facility','wholesale'];
interest=['restaurant','bakery','diner','roof','mobile','bar','diner','breakfast','lounge','grill','tavern']
facility_column=fi_df['facility type'].copy().apply(lambda c: c.lower()).apply(lambda c: any( w in c for w in interest))
fi_df=fi_df[facility_column]
fi_df['facility type'].apply(lambda c: c.lower()).value_counts()


restaurant                                         130213
bakery                                               2851
mobile food dispenser                                 780
mobile food preparer                                  585
golden diner                                          561
                                                    ...  
mobile frozen desserts dispenser-non- motorized         1
deli/bakery                                             1
mobile dessert vendor                                   1
tap room/tavern/liquor store                            1
hooka bar                                               1
Name: facility type, Length: 85, dtype: int64

In [10]:

rules=fi_df['violations'].copy()
rules=rules.dropna().astype(str)

#clean: remove comments and separate multiple violations
def discard_comment(x):
    x=x.strip()
    x=x.split('-')
    return x[0].strip()
rules=rules.apply(lambda v: v.split('|'))
rules=rules.apply(lambda v: [discard_comment(v_i) for v_i in v])
rules=np.unique(rules.explode())

In [11]:
#need to redo indexing so that each rule number is unique, 
rules

array(['1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOWLEDGE, AND PERFORMS DUTIES',
       '1. SOURCE SOUND CONDITION, NO SPOILAGE, FOODS PROPERLY LABELED, SHELLFISH TAGS IN PLACE',
       '10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED AND ACCESSIBLE',
       '10. SEWAGE AND WASTE WATER DISPOSAL, NO BACK SIPHONAGE, CROSS  CONNECTION AND/OR BACK FLOW',
       '11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, DESIGNED, AND MAINTAINED',
       '11. FOOD OBTAINED FROM APPROVED SOURCE',
       '12. FOOD RECEIVED AT PROPER TEMPERATURE',
       '12. HAND WASHING FACILITIES: WITH SOAP AND SANITARY HAND DRYING DEVICES, CONVENIENT AND ACCESSIBLE TO FOOD PREP AREA',
       '13. FOOD IN GOOD CONDITION, SAFE, & UNADULTERATED',
       '13. NO EVIDENCE OF RODENT OR INSECT INFESTATION, NO BIRDS, TURTLES OR OTHER ANIMALS',
       '14. PREVIOUS SERIOUS VIOLATION CORRECTED, 7',
       '14. REQUIRED RECORDS AVAILABLE: SHELLSTOCK TAGS, PARASITE DESTRUCTION',
       '15. FOOD SEPARATED AND PROTECTED',
       

In [12]:
fi_df['violations']

0                                                       NaN
1                                                       NaN
2                                                       NaN
3                                                       NaN
4         55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...
                                ...                        
195443                                                  NaN
195444                                                  NaN
195445    18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
195446                                                  NaN
195447                                                  NaN
Name: violations, Length: 136141, dtype: object

In [13]:
fi_df['inspection date']=pd.to_datetime(fi_df['inspection date'])

In [14]:
years=fi_df['inspection date'].apply(lambda d: d.year).value_counts()
years

2016    16856
2017    16013
2014    15654
2015    14906
2013    14537
2018    12355
2012    11869
2011    11800
2010    11251
2019    10900
Name: inspection date, dtype: int64

In [19]:
fi_df['result'].value_counts()

Pass                    73521
Fail                    25642
Pass w/ Conditions      21045
Out of Business          9286
No Entry                 5289
Not Ready                1332
Business Not Located       26
Name: result, dtype: int64

In [21]:
fi_df['restaurant name DBA'].value_counts()

SUBWAY                      2649
DUNKIN DONUTS               1367
MCDONALD'S                   564
MCDONALDS                    322
CHIPOTLE MEXICAN GRILL       308
                            ... 
BULLSEYE GRILL                 1
PIER 5736                      1
TACOS & SALSAS LLC             1
MAKER'S MARK BARREL ROOM       1
SLICE                          1
Name: restaurant name DBA, Length: 15124, dtype: int64

Now let's start our first visualisation of the chicagoean restaurant map, for this we will be using Folium introduced in the course. Let's start with a very simple one: the restaurants that passed the inspection in 2019.

In [15]:
import folium
loc=[fi_df['latitude'][0],fi_df['longitude'][0]]
m=m = folium.Map(location=loc)
m


In [16]:
#all the inspected 2019 restaurants with result passed
passed_2019=fi_df.copy()
passed_2019=passed_2019[[date.year==2019 for date in passed_2019['inspection date']]]
passed_2019=passed_2019[[result=='Pass' for result in passed_2019['result']]]
passed_2019=passed_2019[[not math.isnan(l)for l in passed_2019['latitude']]]
passed_2019=passed_2019[[not math.isnan(l)for l in passed_2019['longitude']]]
passed_2019.index=range(passed_2019.shape[0])
passed_2019.shape



(2227, 17)

In [17]:

for i in range(20):
    loc=[passed_2019['latitude'][i], passed_2019['longitude'][i]]
    folium.Marker(loc, popup=passed_2019['restaurant name DBA'][i]).add_to(m)

In [18]:
m

2000 markers don't fit into a map, need an alternative